In [47]:
# Defining search parameters (llm used, prompt template)
# the formatting of the documents is also a tunable knob i'd say. Thats why its not taken from the chain.py file but copied here again. We can play around with that formatting if it actually helps our llm if we format it in a specific way. But for now its in a different cell

import prompt_templates
from langchain_community.llms.ollama import Ollama


MODEL = "mistral" # can be replaed by grid search later
prompt_template = prompt_templates.v005 #can be replaced by GS later
file_name = "test_structure.json"


model = Ollama(model=MODEL, format="json")

In [76]:
from langchain_core.documents import Document

def format_documents(docs: [Document]):
    casted_docs = []
    for doc in docs: 
        casted_doc = Document(**doc)
        casted_docs.append(casted_doc)
        
    # Create a dictionary to hold reports for each player ID
    player_reports = defaultdict(list)

    # Aggregate reports by player ID
    for doc in casted_docs:
        player_id = doc.metadata['player_transfermarkt_id']
        report_content = doc.page_content
        player_reports[player_id].append(report_content)

    # Format the aggregated reports
    formatted_reports = []
    for player_id, reports in player_reports.items():
        formatted_report = f"Player ID: {player_id}\n"
        for i, report in enumerate(reports, 1):
            formatted_report += f"Report {i}: {report}\n"
        formatted_report += "###"
        formatted_reports.append(formatted_report.strip())

    # Join all formatted reports into a single string
    return_string = "\n\n".join(formatted_reports)
    #print("------------\nAfter merging reports for each player:\n")
    #print(return_string)
    return return_string

In [49]:
# Define model

from dataclasses import dataclass
from langchain_core.documents import Document
from typing import List

@dataclass(init=True)
class QueryAndRetrievedDocuments:
    query: str
    retrieved_documents: List[Document]
            

@dataclass
class DataModel:
    data: List[QueryAndRetrievedDocuments]

In [78]:
from collections import defaultdict
import json

# Create the context
def load_inputs() -> DataModel:
    with open(file_name, "r") as file:
        json_data = file.read()
        parsed_data = json.loads(json_data)     
        return parsed_data
    

In [79]:
inputs: DataModel = load_inputs()
print(inputs[0])

{'query': 'my first query', 'retrieved_documents': [{'page_content': 'First page content!', 'metadata': {'id': 1, 'player_id': '12', 'player_transfermarkt_id': '123', 'scout_id': '0', 'grade_rating': 9.5, 'grade_potential': 10.0, 'main_position': 'centralmidfield', 'played_position': 'centralmidfield'}}, {'page_content': 'Second page content!', 'metadata': {'id': 1, 'player_id': '12', 'player_transfermarkt_id': '123', 'scout_id': '0', 'grade_rating': 9.5, 'grade_potential': 10.0, 'main_position': 'centralmidfield', 'played_position': 'centralmidfield'}}]}


In [52]:
print(inputs)

[{'query': 'my first query', 'retrieved_documents': [{'page_content': 'First page content!', 'metadata': {'id': 1, 'player_id': '12', 'player_transfermarkt_id': '123', 'scout_id': '0', 'grade_rating': 9.5, 'grade_potential': 10.0, 'main_position': 'centralmidfield', 'played_position': 'centralmidfield'}}]}]


In [80]:
# for all contexts format the documents so it conforms to the string passed to llm
# then call llm
# then we have a input/llm response pair
# do all metrics on those
# print result

from dacite import from_dict


for singleInput in inputs:
    d = QueryAndRetrievedDocuments(**singleInput)
    print(d)
    print(type(d))
    
    formatted_string = format_documents(d.retrieved_documents)
    print(formatted_string)
    
    #actual_instance = from_dict(data_class=QueryAndRetrievedDocuments, data=singleInput)
    #print(type(testtt))
    #print(testtt['retrieved_documents'])
    #formatted_input_string = format_documents(input['retrieved_documents'])
    #print(formatted_input_string)


QueryAndRetrievedDocuments(query='my first query', retrieved_documents=[{'page_content': 'First page content!', 'metadata': {'id': 1, 'player_id': '12', 'player_transfermarkt_id': '123', 'scout_id': '0', 'grade_rating': 9.5, 'grade_potential': 10.0, 'main_position': 'centralmidfield', 'played_position': 'centralmidfield'}}, {'page_content': 'Second page content!', 'metadata': {'id': 1, 'player_id': '12', 'player_transfermarkt_id': '123', 'scout_id': '0', 'grade_rating': 9.5, 'grade_potential': 10.0, 'main_position': 'centralmidfield', 'played_position': 'centralmidfield'}}])
<class '__main__.QueryAndRetrievedDocuments'>
Player ID: 123
Report 1: First page content!
Report 2: Second page content!
###
